In [2]:
import pandas as pd
import requests
import io

In [17]:
state_info = pd.read_csv('./source_data/state_info.csv', dtype = 'object')
dfs = []

for state in state_info['state_po']:
    r = requests.get(f'https://fred.stlouisfed.org/graph/fredgraph.csv?id=MEHOINUS{state}A672N')
    df = pd.read_csv(io.BytesIO(r.content))
    df['year'] = pd.to_datetime(df['DATE']).dt.year
    df['household_income'] = df[f'MEHOINUS{state}A672N']
    df['state'] = state
    df = df[['year', 'state', 'household_income']]
    df = df.sort_values(by = 'year', ascending = True)

    df_1 = df.groupby('state')[['year', 'household_income']]\
            .apply(lambda table : table.set_index('year').pct_change())\
            .reset_index()\
            .rename(columns = {'household_income':'household_income_1'})

    df_4 = df.groupby('state')[['year', 'household_income']]\
            .apply(lambda table : table.set_index('year').pct_change(4))\
            .reset_index()\
            .rename(columns = {'household_income':'household_income_4'})

    df = pd.merge(left = df_1, right = df_4, how = 'inner', on = 'year')
    df = df[['year', 'state_x', 'household_income_1', 'household_income_4']]\
                    .rename(columns = {'state_x':'state'})

    dfs.append(df)

all_data = pd.concat(dfs, axis = 0, ignore_index = True)
all_data.to_csv('./processed_data/1984-2022-median-household-income.csv', index = False, header = True)